In [1]:

# Import necessary libraries 
from langchain import PromptTemplate, LLMChain 
from langchain.llms import Cohere 
from pydantic import BaseModel 
from typing import Optional 
import wikipediaapi 
 
# Step 1: Set up the Cohere API 
import getpass 
COHERE_API_KEY = getpass.getpass('Enter your Cohere API Key: ') 
cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command") 
 
# Step 2: Download Indian Penal Code (IPC) summary from Wikipedia 
def fetch_ipc_summary(): 
    user_agent = "IPCChatbot/1.0 (contact: kelvinprabhu2071@gmail.com)" 
    wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='en') 
    page = wiki_wiki.page("Indian Penal Code") 
 
    if not page.exists(): 
        raise ValueError("The Indian Penal Code page does not exist on Wikipedia.") 
                                                          
 
 
    return page.text[:5000]  # Limiting to first 5000 characters for brevity 
 
ipc_content = fetch_ipc_summary() 
 
# Step 3: Define a Pydantic model for structured responses 
class IPCResponse(BaseModel): 
    section: Optional[str] 
    explanation: Optional[str] 
 
# Step 4: Create a prompt template for the chatbot 
prompt_template = PromptTemplate( 
    input_variables=["question"], 
    template=""" 
    You are a legal assistant chatbot specialized in the Indian Penal Code (IPC). 
    Refer to the following IPC document content to answer the user's query: 
 
    {ipc_content} 
 
    User Question: {question} 
 
    Provide a detailed answer, mentioning the relevant section if applicable. 
    """ 
) 
 
# Step 5: Function to interact with the chatbot 
def get_ipc_response(question: str) -> IPCResponse:
    formatted_prompt = prompt_template.format(ipc_content=ipc_content, question=question)
    
    try:
        response = cohere_llm.invoke(formatted_prompt)  # updated from predict() to invoke()
    except Exception as e:
        print(f"LLM Error: {e}")
        return IPCResponse(section=None, explanation="Failed to get a response.")
    
    # Extract section intelligently
    if "Section" in response:
        section = response.split('Section')[1].split(':')[0].strip()
        explanation = response.split(':', 1)[-1].strip()
        print(f"Extracted Section: {section}")
        print(f"Extracted Explanation: {explanation}")
    else:
        section = None
        explanation = response.strip()
        print("No specific section mentioned in the response.")
    
    return IPCResponse(section=section, explanation=explanation)

 


/tmp/ipykernel_11375/86711405.py:11: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")


In [5]:
# Step 6: Set up interactive chatbot in Jupyter 
from IPython.display import display 
import ipywidgets as widgets 
 
# Function to display chatbot responses 
def display_response(response: IPCResponse): 
    print(f"Section: {response.section if response.section else 'N/A'}") 
    print(f"Explanation: {response.explanation}") 
 
# Function to handle user input 
def on_button_click(b): 
    user_question = text_box.value 
    print(f"User: {user_question}")
    try: 
        response = get_ipc_response(user_question) 
        display_response(response) 
    except Exception as e: 
        print(f"Error: {e}") 
                                                          
 
 
# Create text box and button for user input 
text_box = widgets.Text( 
    value='', 
    placeholder='Ask about the Indian Penal Code', 
    description='You:', 
    disabled=False 
) 
button = widgets.Button( 
    description='Ask', 
    disabled=False, 
    button_style='', 
    tooltip='Click to ask a question about IPC', 
    icon='legal' 
) 
 
button.on_click(on_button_click) 
 
# Display the chatbot interface 
display(text_box, button)

Text(value='', description='You:', placeholder='Ask about the Indian Penal Code')

Button(description='Ask', icon='legal', style=ButtonStyle(), tooltip='Click to ask a question about IPC')

In [4]:
# Simple chatbot loop using basic input/output

def display_response(response: IPCResponse):
    print("\n--- Chatbot Response ---")
    print(f"Section: {response.section if response.section else 'N/A'}")
    print(f"Explanation: {response.explanation}\n")

def start_ipc_chatbot():
    print("Welcome to the IPC Legal Assistant Chatbot!")
    print("Type 'exit' to quit.\n")

    while True:
        user_question = input("You: ")
        if user_question.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break
        try:
            response = get_ipc_response(user_question)
            display_response(response)
        except Exception as e:
            print(f"Error: {e}\n")

# Start the chatbot
start_ipc_chatbot()


Welcome to the IPC Legal Assistant Chatbot!
Type 'exit' to quit.

Extracted Section: 323
Extracted Explanation: 1. Section 323: If any individual voluntarily causes hurt or commits an act that would be likely to cause hurt, they shall be punished with imprisonment for a term of up to one year, a fine, or both.
2. Section 324: When a person intentionally causes harm by an act that ends in the permanent disability of any limb, organ, or both, or causes an injury that would be likely to cause such permanent disability, the accused shall be punished with imprisonment for a term which may extend to ten years, a fine, or both. 

In addition to these, there are other more severe sections pertaining to more brutal attacks such as sections for voluntarily causing grievous hurt (sections 326-326A) and intentionally causing grievous hurt (sections 327-327A). 

If you feel threatened by any means, it is always a good idea to seek legal advice from a qualified professional or authority.

--- Chatbo